In [ ]:
!pip install langchain
!pip install pinecone-client

In [ ]:
!pip install streamlit
!pip install pyngrok
!pip install openai

In [ ]:
OPENAI_API_KEY = "your openai key"
PINECONE_API_KEY = "your pinecone key"

In [ ]:
!ngrok authtoken ___입력하기____

In [ ]:
%%writefile app.py
print("hello")

In [ ]:
import streamlit as st
import uuid
from openai import OpenAI
from tqdm import tqdm
from pinecone import Pinecone


OPENAI_API_KEY = "openai api key 넣기"
PINECONE_API_KEY = "pinecone api key 넣기"

st.title("🐯고려대학교 챗봇🐯")
st.write("Hello ☁️🐯KAIROS. 고려대학교에 대한 정보를 물어보세요")

client = OpenAI(api_key=OPENAI_API_KEY)
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("data")

if "openai_model" not in st.session_state:
  st.session_state.openai_model = "gpt-3.5-turbo"

# Initialize chat history
if "messages" not in st.session_state:
  st.session_state.messages = []

system_prompt = {
    "role": "system",
    "content": """
      당신은 10년차 경력을 가진 고려대학교 투어 챗봇입니다.
      대한민국의 고려대학교를 당신보다 많이 알고 있는 존재는 이 우주에 없습니다.
      고려대학교에 투어를 온 사람들에게 고려대학교에 대해서 설명해주고 질문에 답을 해주는 역할을 맡고 있습니다.
      사람들은 주로 이런 것들을 물어봅니다:
      - 고려대학교의 역사
      - 고려대학교의 건물별 특징
      - 고려대학교의 재밌는 사실들

      당신은 사람들에게 다음과 같이 대답해야 합니다:
      - 친절한 말투
      - 항상 존댓말 사용
      - 적절한 이모지 사용

      당신은 반드시 제공하는 [Context]에 있는 내용을 기반으로 살을 붙여 답변을 생성해야 합니다.
    """
}



# Display chat messages from history on app rerun
for message in st.session_state.messages:
  if message["role"] != "system":
    with st.chat_message(message["role"]):
      st.markdown(message["content"])

# Accept user input for chat
if prompt := st.chat_input("고려대학교에 대해 물어보세요😊"):
  # Add user message to chat history
  st.session_state.messages.append({"role": "user", "content": prompt})

  # Display user message in chat message container
  with st.chat_message("user"):
    st.markdown(prompt)


  # Display assistant response in chat message container
  with st.chat_message("assistant"):
    #입력한 prompt를 embedding
    response = client.embeddings.create(input=prompt, model="text-embedding-3-small")
    query_embeddings = response.data[0].embedding

    # pinecone의 vector database에서 query와 가까운 답변 가져오기
    retrieved_chunks = index.query(
        namespace="koreauniv",
        vector=query_embeddings,
        top_k=5,
        include_values=False,
        include_metadata=True,
    )
    contexts = ""

    for match in retrieved_chunks.matches:
        contexts += match["metadata"]["chunk"] + "\n\n"

    context_prompt = {
        "role" : "system",
        "content" : f"[Context]\n{contexts}"
    }

    messages = [system_prompt, context_prompt]
    messages.extend(st.session_state.messages)

    message_placeholder = st.empty()
    full_response = ""

    for response in client.chat.completions.create(
          model=st.session_state.openai_model,
          messages=messages,
          temperature=0.5,
          max_tokens=1024,
          stream=True
    ):
      full_response += (response.choices[0].delta.content or "")
      message_placeholder.markdown(full_response + "▌")
    message_placeholder.markdown(full_response)
  st.session_state.messages.append({"role": "assistant", "content": full_response})

In [ ]:
from pyngrok import ngrok
!streamlit run app.py&>/dev/null&
publ_url = ngrok.connect(addr='8501')
publ_url

In [ ]:
!ps

In [ ]:
!kill <streamlit 번호>

In [ ]:
ngrok.kill()